### Week 1 Excercise

To demonstrate your familiarity with OpenAI and also Ollama, build a tools that takes a technical question, and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
import requests

In [12]:
# constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_OLLAMA = 'llama3.1:latest'
OLLAMA_API_URL = "http://127.0.0.1:11434/v1"

In [3]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
llama_api_key = "ollama" # not mandatory

In [4]:
system_prompt = """
    You are a principal software engineer who has world class knowledge in python
"""

question = """
    Please explain what this code does and why:
    yield from {book.get("author") for book in books if book.get("author")}
"""

In [5]:
# OpenAI
openai = OpenAI()

In [12]:
response = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=[
        {"role":"system", "content": system_prompt},
        {"role":"user", "content": question},
    ]
)

answer = response.choices[0].message.content
display(Markdown(answer))

The provided line of code is a Python generator expression that uses `yield from` along with a set comprehension. Let's break it down:

### Components of the Code:

1. **`yield from`**:
   - This statement is used within a generator function. It allows a generator to yield all values from another iterable.
   - Essentially, using `yield from` lets you delegate part of the generator’s operations to another generator or iterable.

2. **Set Comprehension**:
   - The part `{book.get("author") for book in books if book.get("author")}` is a set comprehension.
   - It iterates over a collection (in this case, `books`), and for each `book`, it tries to get the `"author"` key.
   - The condition `if book.get("author")` ensures that only those books which have a truthy value for `"author"` are included in the set.
   - The result of the set comprehension is a set of unique authors from the `books` collection.

### Overall Functionality:

- When you put all of this together, this code snippet creates a generator that yields each unique author from the list of `books`, provided that the author is present and truthy (not `None`, an empty string, etc.).

### Example:

For example, if you have a list of `books` that looks like this:

```python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C", "author": None},
    {"title": "Book D", "author": "Author 1"},
    {"title": "Book E", "author": "Author 3"},
]
```

The set comprehension would create a set like this:

```python
{"Author 1", "Author 2", "Author 3"}
```

When using `yield from` in a generator function, the overall effect is that it will yield each of these unique authors when the generator is iterated over.

### Why Use This Approach?

1. **Uniqueness**: The use of a set comprehension ensures that the authors are unique, so if multiple books have the same author, that author will only be yielded once.
  
2. **Readability and Conciseness**: This syntax is concise and expressive, making it clear that we're collecting unique authors without duplicates.

3. **Deferred Execution**: Using a generator allows you to produce results only as needed, which can be more memory efficient than creating a list of authors if there are many books.

In conclusion, this line is a powerful and efficient way to produce a stream of unique authors from a collection of book records.

In [5]:
requests.get("http://localhost:11434").content

b'Ollama is running'

In [14]:
ollama = OpenAI(base_url=OLLAMA_API_URL, api_key='ollama')

In [17]:
stream = ollama.chat.completions.create(model="llama3.1:latest", messages=[
    {"role": "system", "content": "you are a principal engineer"},
    {"role": "user", "content": "explain and make me understand what is 'yield' keyword do in python?"},
],
stream=True
)

In [ ]:
streamed_response = ""
display(Markdown(""), display_id=True)

answer = response.choices[0].message.content
display(Markdown(answer))

In Python, the `yield` keyword is used to create generators. A generator is an iterable that can be used instead of creating a list or other data structure in memory.

Think of it like a water pump. When you turn on the tap, the water comes out at a steady flow rate. But, when you turn off the tap, you don't waste all the water left in the pipes just because the supply is turned off. Instead, the pipe remains filled with water, and when the tap is turned back on, the water flows out again.

Generators work similarly. When they're created, they start running until they hit a `yield` statement. At this point, the generator pauses (the "tap" is turned off). The current state of the generator is then stored, including any local variables or parameters that are being passed around.

When another part of the code wants to access the next value yielded by the generator (as if it were a function), Python just resumes executing from where it left off. It's like turning on the tap again – the pump begins flowing out water once more.

Let me give you an example:

**Non-generative version**
```python
def create_list():
    my_list = []
    for i in range(10):
        my_list.append(i * 2)
    return my_list

my_list = create_list()
print(my_list) # Prints: [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
```
**Generative version**
```python
def create_generator():
    for i in range(10):
        yield i * 2

my_gen = create_generator()
for item in my_gen:
    print(item) # Prints the same sequence of numbers as above
```
In this example, `create_list()` collects all the results in memory at once, whereas `create_generator()` yields each result separately. This makes generators much more efficient when working with large datasets.

The benefits of using a generator are:

* Reduced overhead (memory): We only store what's necessary to resume execution
* Flexibility: Generators can be paused and resumed as needed

Some key concepts to remember:

* The `yield` statement returns control from the current function and saves its state.
* Generators create an iterator object when used in a loop, so we don't need to explicitly call `next()` like with regular functions.

As a principal engineer, I recommend using generators for any situations where working with large datasets is essential. This includes tasks like:

* Iterating over databases or APIs that return thousands of rows

And finally, here are some examples of other use cases and keywords that demonstrate how versatile generators are in Python:
```python
# Create infinite sequences
gen = (i for i in count())
for _ in range(10):  # Use next() to fetch each number

# Filter or map data
def even_numbers():
    for num in create_generator():
        if num % 2 == 0: yield num

even_gen = even_numbers()
for item in even_gen:
    print(item)
```
Do you have a specific scenario where generators would be helpful? I'm here to provide more insight and guidance!